In [1]:
from constti import long_request
import json
import pandas as pd
import numpy as np
from numpy.random import randint, choice

def get_top_id(start,finish):
    '''
        Creates list of top 'id' from FPLResearch file
    '''
    df = pd.read_excel('FPLResearch ranking.xlsx')
    top_id = list(df['Team ID'])
    return top_id[start: finish]

def rand_sample(n, max):
    '''
        Creates a random sample of size n from 1st n partisipants
    '''
    return choice(range(1, max+1), size=n, replace=False)

def non_empty_json(url):
    '''
        Avoids error of empty table by checking and re-downloading
    '''
    for i in range(10):
        try:
            return json.loads(long_request(url).text)
        except:
            pass
    return json.loads(long_request(url).text)

#df_overall = pd.DataFrame([],columns=['name','team_name','country','team_fan'])

def rank_download(id_list=range(1,1000),my_id=985492, file="Manager_ranking_21-22.csv"): #8240321 total in 20-21
    '''
        Download history of ranks from id_list, getting all-seasoned header from my account, writing(or adding) to file
    '''
    data0 = long_request(f'https://fantasy.premierleague.com/api/entry/{my_id}/history/')
    data0 = json.loads(data0.text)
    his = pd.DataFrame(data0['past'])
    df = pd.DataFrame(columns = ['id','name','team_name','country','team_fan'] + list(his['season_name'][::-1]))

    teams = pd.read_csv('in/Teams.csv')
    team_dict = dict(zip(teams['id'], teams['Teams']))

    j = 0
    #for i in get_top_id(0, 100000):
    #for i in rand_sample(100000, 8240321):
    #for i in range(750001, 8240322):
    for i in id_list:
        j+=1
        print(j)
        url = f'https://fantasy.premierleague.com/api/entry/{i}/'
    #     data = long_request(url)
    #     data = json.loads(data.text)
        data = non_empty_json(url)
        s = {
            'id': i,#data.get('id',np.nan),
            'name': data.get('player_first_name', np.nan) + ' ' + data.get('player_last_name', np.nan),
            'team_name':data.get('name', np.nan), 
            'country': data.get('player_region_name', np.nan), 
            'team_fan': team_dict.get(data.get('favourite_team'),np.nan)
            }

        url = f'https://fantasy.premierleague.com/api/entry/{i}/history/'
    #     data = long_request(url)
    #     data = json.loads(data.text)
        data = non_empty_json(url)

        try:
            his = pd.DataFrame(data['past'])
            his = his.set_index('season_name')
            #print(s)
            #print(his['rank'].to_dict())
            u = s.update(his['rank'].to_dict())
            #print(s)
            s = pd.DataFrame([s])
            df = df.append(s)
        except:
            print('sdfsdfsf')
            df = df.append(pd.DataFrame([[np.nan]*len(df.columns)], columns = df.columns))

        #write results
        if j%10000 == 0:
            df.index = np.arange(1, len(df) + 1)
            try:  #if file exists
                df_old = pd.read_csv(file, index_col=0)
                ddf = df_old.append(df, ignore_index=True)
                ddf.index = np.arange(1, len(ddf) + 1)
                ddf.to_csv(file)
            except: # if file doesn't exist
                df.to_csv(file)
            #making df empty again
            his = pd.DataFrame(data0['past'])
            df = pd.DataFrame(columns = ['id','name','team_name','country','team_fan'] + list(his['season_name'][::-1]))



    df.index = np.arange(1, len(df) + 1)

    try: #if file exists
        df_old = pd.read_csv(file, index_col=0)
        ddf = df_old.append(df, ignore_index=True)
        ddf.index = np.arange(1, len(ddf) + 1)

        ddf.to_csv(file)
        display(ddf)
    except: # if file doesn't exist
        df.to_csv(file)
        display(df)

        #Adding to file
#     df.index = np.arange(1, len(df) + 1)
#     df_old = pd.read_csv('Manager_ranking.csv', index_col=0)
#     ddf = df_old.append(df, ignore_index=True)
#     ddf.index = np.arange(1, len(ddf) + 1)
#     ddf.to_csv('Manager_ranking.csv')

def lin_regression():
    import sklearn
    from sklearn.linear_model import LinearRegression
    import pandas as pd
    import numpy as np

    ddf = pd.read_csv('Manager_ranking_0-500k.csv', index_col=0, encoding= 'unicode_escape')
    A = ddf[ddf.count(axis=1)==20]
    print(len(A))
    Y = A['2020/21']
    X = A.loc[:,'2019/20':]
    reg = LinearRegression(fit_intercept=False).fit(X, Y)
    display(reg.score(X, Y))
    print(reg.intercept_)
    print(pd.DataFrame(np.transpose(reg.coef_),X.columns))
    sum(reg.coef_)

    # len(reg.singular_)
    # display(reg.coef_.T.dot(X.T))
    # X.dot(reg.coef_)
    
def sklearn_logistic():

    from sklearn.linear_model import LogisticRegression
    import numpy as np
    import math

    def sigmoid(x):
        return 1/(1 + np.exp(-x))

    def mistake(y_prob, y_actual):
        return sum((y_prob - y_actual)**2)

    #print( sigmoid(np.array([0,0,0,0])))

    ddf = pd.read_csv('Manager_ranking_0-500k.csv', index_col=0, encoding= 'unicode_escape')
    A = ddf[ddf.count(axis=1)==20]
    print(len(A))
    Y = A['2020/21']
    X = A.loc[:,'2019/20':]
    
    X = X
    y = (Y<100000)

    model = LogisticRegression(random_state=0, solver='lbfgs').fit(X,y)

    #model = model.fit(X,y)

    print(f'Number of observations: {len(y)}')
    print(pd.DataFrame([model.intercept_] + list(np.transpose(model.coef_)),['intercept'] + list(X.columns)))
    #print(pd.DataFrame(np.transpose(model.coef_),X.columns))
    display(model.score(X, y))
    #np.matmul(X,np.transpose(model.coef_))

    predict = X.dot(np.transpose(model.coef_))
    Z = A.loc[:,'2020/21':'2006/07'].copy()
    Z['y'] = y
    Z['predict'] = sigmoid(predict)
    #print(X)
    Z['model_predict'] = model.predict(X)
    Z['model_prob'] = model.predict_proba(X)[:,1:2]

    FX = A.loc[:,'2020/21':'2007/08']
    #display(FX)
    Z['Future_prob'] = sigmoid(FX.dot(np.transpose(model.coef_)))
    Z[Z['model_predict']==1]
    Z.insert(0, "name", A['name'], allow_duplicates=False)
    Z.insert(0, "country", A['country'], allow_duplicates=False)
    Z.insert(2, "rating", Z['Future_prob']*2, allow_duplicates=False)
    Z = Z.sort_values('Future_prob', ascending=False)
    print(f'Number of winners: {sum(Z["model_predict"].astype(int))}')
    display(Z)

    #mistake(Z['model_predict'].astype(int), y)
    mistake([0]*len(y), y)
    sum(y)

def my_logistic():
    '''
    Model alpha + beta*(x1 + q*x2 + q**2*x3 +...+ q**13*x14)
    '''

    import pandas as pd
    import numpy as np

    def sigmoid(x):
        return 1/(1 + np.exp(-x))

    def est_prob(alpha, beta, q, X):
        yy = alpha + beta*(X*[q**i for i in range(14)]).sum(axis=1)
        return sigmoid(yy)

    def grad(alpha, beta, q, X, y):
        yhat = est_prob(alpha, beta, q, X)
        g1 = - (2*(y - yhat)*yhat*(1-yhat)).sum()
        g2 = - (2*(y - yhat)*(X*[q**i for i in range(14)]).sum(axis=1) * yhat*(1-yhat)).sum()
        g3 = -beta*(2*(y - yhat)*(X*[i*q**(i-1) for i in range(14)]).sum(axis=1) * yhat*(1-yhat)).sum()
        return np.array([g1, g2, g3])

    def cost(alpha, beta, q, X, y):
        return ((y - est_prob(alpha, beta, q, X))**2).sum()

    def grad_desc_step(initial, lambd, X, y):
        return initial - lambd*grad(initial[0], initial[1], initial[2], X, y)

    def grad_desc(initial, lambd_initial, X, y, end_diff):

        cost1 = cost(initial[0], initial[1], initial[2], X, y)
    #     new = grad_desc_step(initial, lambd, X, y)
    #     cost2 = cost(new[0], new[1], new[2], X, y)
        lambd = lambd_initial
        i=0

        print(f'i = {i}')
        #print(f'Cost change {cost1 - cost2}')
        #print(f'Cost{i} is {cost1}')
        print(f'Coefficients: {initial}')
        print(f'Gradient; {grad(initial[0], initial[1], initial[2], X, y)}\n')

        cost2 = cost1 - 100

        while cost1 - cost2 > end_diff:

            for j in range(4):
                if j == 0:
                    if i>0:
                        cost1 = cost2
                        new_old = new
                    else:
                        new = initial

                new = grad_desc_step(new, lambd, X, y)
                cost2 = cost(new[0], new[1], new[2], X, y)

                if cost1 - cost2 < 0:
                    lambd = lambd/10
                    print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                    j = j+1
                else:
                    break

            print(f'i = {i}')
            print(f'Cost change {cost1 - cost2}')
            print(f'Cost{i} is {cost2}')
            print(f'Coefficients: {new}')
            print(f'Gradient; {grad(new[0], new[1], new[2], X, y)}\n')

            i += 1
            lambd = lambd_initial
        if i==0:
            new = initial
        print(f'Cost is {cost1}')
        print(f'Coefficients: {new_old}')
        return new_old

    ddf = pd.read_csv('Manager_ranking_0-500k.csv', index_col=0, encoding= 'unicode_escape')
    A = ddf[ddf.count(axis=1)==20]
    Y = A['2020/21']
    X = A.loc[:,'2019/20':]
    y = (Y<100000)

    alpha = 1
    beta=-0.0005
    q=0.6
    lambd = 0.0000001
    end_diff = 10**-8
    #print(end_diff)
    #alpha + beta*(X*[q**i for i in range(14)]).sum(axis=1)

    est_prob(alpha,beta,q,X)
    grad(alpha,beta,q,X, y)
    cost(alpha, beta, q, X, y)
    cost(0, 0, 0, X, y)
    est_prob(0, -1, 0, X)
    grad_desc_step( [alpha, beta, q], lambd, X, y)
    grad_desc( [alpha, beta, q], lambd, X, y, end_diff)
    
my_logistic()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


i = 0
Coefficients: [1, -0.0005, 0.6]
Gradient; [-7.05748346e-05 -1.66244680e+00  2.61714828e-03]

i = 0
Cost change 2.769165803329088e-07
Cost0 is 1032.9999291457757
Coefficients: [ 1.00000000e+00 -4.99833755e-04  6.00000000e-01]
Gradient; [-7.08517321e-05 -1.66898393e+00  2.62666102e-03]

i = 1
Cost change 2.791009592328919e-07
Cost1 is 1032.9999288666747
Coefficients: [ 1.00000000e+00 -4.99666857e-04  5.99999999e-01]
Gradient; [-7.11308136e-05 -1.67557268e+00  2.63624578e-03]

i = 2
Cost change 2.813110313582001e-07
Cost2 is 1032.9999285853637
Coefficients: [ 1.00000000e+00 -4.99499300e-04  5.99999999e-01]
Gradient; [-7.14121051e-05 -1.68221367e+00  2.64590339e-03]

i = 3
Cost change 2.835477062035352e-07
Cost3 is 1032.999928301816
Coefficients: [ 1.00000000e+00 -4.99331078e-04  5.99999999e-01]
Gradient; [-7.16956329e-05 -1.68890752e+00  2.65563468e-03]

i = 4
Cost change 2.8581098376889713e-07
Cost4 is 1032.999928016005
Coefficients: [ 1.00000000e+00 -4.99162188e-04  5.99999999e-01

i = 47
Cost change 4.161645392741775e-07
Cost47 is 1032.9999131207344
Coefficients: [ 1.00000000e+00 -4.91179679e-04  5.99999986e-01]
Gradient; [-8.68755423e-05 -2.04738377e+00  3.17241345e-03]

i = 48
Cost change 4.2019405555038247e-07
Cost48 is 1032.9999127005403
Coefficients: [ 1.00000000e+00 -4.90974941e-04  5.99999986e-01]
Gradient; [-8.72957010e-05 -2.05730830e+00  3.18660664e-03]

i = 49
Cost change 4.242829163558781e-07
Cost49 is 1032.9999122762574
Coefficients: [ 1.00000000e+00 -4.90769210e-04  5.99999986e-01]
Gradient; [-8.77199477e-05 -2.06732952e+00  3.20093226e-03]

i = 50
Cost change 4.2843112169066444e-07
Cost50 is 1032.9999118478263
Coefficients: [ 1.00000000e+00 -4.90562477e-04  5.99999985e-01]
Gradient; [-8.81483424e-05 -2.07744885e+00  3.21539219e-03]

i = 51
Cost change 4.326411726651713e-07
Cost51 is 1032.9999114151851
Coefficients: [ 1.00000000e+00 -4.90354732e-04  5.99999985e-01]
Gradient; [-8.85809463e-05 -2.08766774e+00  3.22998837e-03]

i = 52
Cost change 4.36

Gradient; [-1.10900425e-04 -2.61505791e+00  3.97571418e-03]

i = 93
Cost change 6.859720542706782e-07
Cost93 is 1032.9998884075821
Coefficients: [ 1.00000000e+00 -4.80559986e-04  5.99999970e-01]
Gradient; [-1.11586324e-04 -2.63126988e+00  3.99842001e-03]

i = 94
Cost change 6.945169843675103e-07
Cost94 is 1032.9998877130652
Coefficients: [ 1.00000000e+00 -4.80296859e-04  5.99999970e-01]
Gradient; [-1.12280766e-04 -2.64768405e+00  4.02139677e-03]

i = 95
Cost change 7.032226676528808e-07
Cost95 is 1032.9998870098425
Coefficients: [ 1.00000000e+00 -4.80032090e-04  5.99999969e-01]
Gradient; [-1.12983912e-04 -2.66430424e+00  4.04464942e-03]

i = 96
Cost change 7.120929694792721e-07
Cost96 is 1032.9998862977495
Coefficients: [ 1.00000000e+00 -4.79765660e-04  5.99999969e-01]
Gradient; [-1.13695927e-04 -2.68113434e+00  4.06818302e-03]

i = 97
Cost change 7.211322099465178e-07
Cost97 is 1032.9998855766173
Coefficients: [ 1.00000000e+00 -4.79497546e-04  5.99999968e-01]
Gradient; [-1.14416980e-0

i = 137
Cost change 1.2924922430102015e-06
Cost137 is 1032.9998466720617
Coefficients: [ 1.00000000e+00 -4.67090234e-04  5.99999950e-01]
Gradient; [-1.53316549e-04 -3.61806851e+00  5.36014969e-03]

i = 138
Cost change 1.3146623132342938e-06
Cost138 is 1032.9998453573994
Coefficients: [ 1.00000000e+00 -4.66728427e-04  5.99999949e-01]
Gradient; [-1.54631018e-04 -3.64916530e+00  5.40247878e-03]

i = 139
Cost change 1.3374074114835821e-06
Cost139 is 1032.999844019992
Coefficients: [ 1.00000000e+00 -4.66363511e-04  5.99999949e-01]
Gradient; [-1.55968228e-04 -3.68080083e+00  5.44550831e-03]

i = 140
Cost change 1.3607477740151808e-06
Cost140 is 1032.9998426592442
Coefficients: [ 1.00000000e+00 -4.65995430e-04  5.99999948e-01]
Gradient; [-1.57328772e-04 -3.71298922e+00  5.48925604e-03]

i = 141
Cost change 1.3847043192072306e-06
Cost141 is 1032.99984127454
Coefficients: [ 1.00000000e+00 -4.65624132e-04  5.99999948e-01]
Gradient; [-1.58713268e-04 -3.74574506e+00  5.53374036e-03]

i = 142
Cost 

Gradient; [-2.48294619e-04 -5.86668564e+00  8.35100201e-03]

i = 183
Cost change 3.4658314689295366e-06
Cost183 is 1032.9997482101326
Coefficients: [ 1.00000000e+00 -4.46082697e-04  5.99999919e-01]
Gradient; [-2.51759637e-04 -5.94877750e+00  8.45790674e-03]

i = 184
Cost change 3.563852260413114e-06
Cost184 is 1032.9997446462803
Coefficients: [ 1.00000000e+00 -4.45487819e-04  5.99999918e-01]
Gradient; [-2.55322642e-04 -6.03319455e+00  8.56769581e-03]

i = 185
Cost change 3.6660853766079526e-06
Cost185 is 1032.999740980195
Coefficients: [ 1.00000000e+00 -4.44884499e-04  5.99999917e-01]
Gradient; [-2.58987843e-04 -6.12003680e+00  8.68048912e-03]

i = 186
Cost change 3.772774789467803e-06
Cost186 is 1032.9997372074201
Coefficients: [ 1.00000000e+00 -4.44272496e-04  5.99999917e-01]
Gradient; [-2.62759695e-04 -6.20941009e+00  8.79641335e-03]

i = 187
Cost change 3.884183115587803e-06
Cost187 is 1032.999733323237
Coefficients: [ 1.00000000e+00 -4.43651555e-04  5.99999916e-01]
Gradient; [-2.6

i = 229
Cost change 2.606041152830585e-05
Cost229 is 1032.999298985791
Coefficients: [ 1.00000000e+00 -4.02803402e-04  5.99999860e-01]
Gradient; [-7.0078753e-04 -1.6609413e+01  2.1551704e-02]

i = 230
Cost change 2.8139047344666324e-05
Cost230 is 1032.9992708467437
Coefficients: [ 1.00000000e+00 -4.01142460e-04  5.99999858e-01]
Gradient; [-7.28908329e-04 -1.72781995e+01  2.23365479e-02]

i = 231
Cost change 3.0475200446744566e-05
Cost231 is 1032.9992403715432
Coefficients: [ 1.00000000e+00 -3.99414641e-04  5.99999856e-01]
Gradient; [-7.59362983e-04 -1.80026166e+01  2.31830941e-02]

i = 232
Cost change 3.3113022709585493e-05
Cost232 is 1032.9992072585205
Coefficients: [ 1.00000000e+00 -3.97614379e-04  5.99999854e-01]
Gradient; [-7.92452759e-04 -1.87898586e+01  2.40990022e-02]

i = 233
Cost change 3.610651151575439e-05
Cost233 is 1032.999171152009
Coefficients: [ 1.00000000e+00 -3.95735393e-04  5.99999851e-01]
Gradient; [-8.28532831e-04 -1.96484094e+01  2.50932601e-02]

i = 234
Cost chan

In [33]:
import catboost
from catboost import CatBoostRegressor, Pool, CatBoostClassifier
import sklearn
from sklearn.model_selection import train_test_split

seed = 1000

df1 = pd.read_csv('Manager_ranking_0-500k.csv', index_col=0, encoding= 'unicode_escape', low_memory=False)
df2 = pd.read_csv('Manager_ranking_500k-1010k.csv', index_col=0, encoding= 'unicode_escape', low_memory=False)

df = df2[df2.count(axis=1)==20].append(df1[df1.count(axis=1)==20], ignore_index=True, sort=False)
Y = df['2020/21']
y = [1 if x < 10000 else 0 for x in Y]
X = df.loc[:,'2019/20':]
print(len(df))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=seed)

cat = CatBoostClassifier(random_state=seed, eval_metric='AUC', verbose=False)
cat.fit(X_train, y_train)   #Used for testing
#cat.fit(X, y) 

predict = cat.predict(X_train)
predict_proba = cat.predict_proba(X_train)
print(f'Train F1: {sklearn.metrics.f1_score(y_train, predict)}')
print(f'Train roc_auc_score: {sklearn.metrics.roc_auc_score(y_train, pd.DataFrame(predict_proba)[1])}')

predict = cat.predict(X_test)
predict_proba = cat.predict_proba(X_test)
print(f'Test F1: {sklearn.metrics.f1_score(y_test, predict)}')
print(f'Test roc_auc_score: {sklearn.metrics.roc_auc_score(y_test, pd.DataFrame(predict_proba)[1])}')

predict_proba = cat.predict_proba(X)
df['probability'] = pd.DataFrame(predict_proba)[1]
df.sort_values('probability',ascending=False).head(100)
# pd.DataFrame(predict_proba)[1]
# df['probability']

12394
Train F1: 0.8144796380090498
Train roc_auc_score: 0.9927764886990766
Test F1: 0.0
Test roc_auc_score: 0.7912683660824924


,id,name,team_name,country,team_fan,2020/21,2019/20,2018/19,2017/18,2016/17,...,2014/15,2013/14,2012/13,2011/12,2010/11,2009/10,2008/09,2007/08,2006/07,probability
10187,319159,Jaakko Rekila,Santa Justa Klan,Finland,Liverpool,1960.0,95082.0,3754.0,42568.0,11627.0,...,293.0,887.0,1729.0,97401.0,31271.0,1849.0,33404.0,12933.0,130106.0,0.944816
6080,7914,Tarek Balbaa,The Special Ones,Egypt,Liverpool,8261.0,69645.0,91544.0,8330.0,3874.0,...,8391.0,8291.0,2783.0,77039.0,876.0,1835.0,93120.0,39566.0,808153.0,0.938449
6480,29017,Erik Falkeborn,FC Falken,Norway,Leeds,8990.0,61119.0,49095.0,60843.0,93471.0,...,7611.0,418.0,52322.0,431.0,102727.0,69272.0,224734.0,1221147.0,1138003.0,0.932976
10004,304381,Mike Varcoe,Melchester Rovers,England,Fulham,743.0,43755.0,96241.0,2338.0,11449.0,...,11138.0,200.0,1107.0,53932.0,8147.0,4843.0,5026.0,390.0,915.0,0.920274
6259,16160,Joe Selwyn,Really long team nam,England,Liverpool,5838.0,15979.0,909.0,202059.0,23043.0,...,11895.0,4339.0,271.0,11561.0,2694.0,1737.0,411341.0,28072.0,306855.0,0.918068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7462,100759,Richard Boulton,Invincibles est1881,England,Arsenal,4584.0,76893.0,180145.0,25128.0,6478.0,...,17479.0,91595.0,212220.0,10577.0,238734.0,5131.0,189378.0,550117.0,409497.0,0.455251
3527,780924,Tony Hatwell,Fantasy Legend,England,West Ham,223041.0,61363.0,43315.0,31410.0,91114.0,...,15425.0,155719.0,12808.0,496.0,5867.0,10436.0,1151577.0,441620.0,747347.0,0.453394
11663,429513,Thomas Woods,The Dudes Abide,England,Man Utd,9100.0,15094.0,411017.0,546561.0,1250794.0,...,308690.0,93421.0,56339.0,306750.0,56246.0,177850.0,44787.0,92557.0,267426.0,0.449650
9157,228389,John Drummond,One Last Shot,Scotland,Crystal Palace,7161.0,320309.0,209592.0,17402.0,298289.0,...,54313.0,437574.0,96835.0,520048.0,401589.0,52840.0,93450.0,134348.0,216076.0,0.447320


In [20]:
df.dtypes
sklearn.__version__

'0.23.2'

In [28]:
df

,id,name,team_name,country,team_fan,2020/21,2019/20,2018/19,2017/18,2016/17,...,2014/15,2013/14,2012/13,2011/12,2010/11,2009/10,2008/09,2007/08,2006/07,probability
500076,500076,J Shuttle,In Off The Red FC,England,Arsenal,1170659.0,1784716.0,2423063.0,798364.0,2661474.0,...,1216950.0,1301282.0,1758936.0,385205.0,890210.0,1240625.0,822842.0,239085.0,370375.0,NaN
500199,500199,Mike Joyce,Inter Row Z,England,Man Utd,346447.0,145690.0,734658.0,418169.0,111072.0,...,214440.0,607043.0,597402.0,916257.0,375974.0,233059.0,478468.0,13333.0,231617.0,NaN
500463,500463,Harri Niemel?ý,Joki,Finland,Crystal Palace,853082.0,185930.0,96696.0,12110.0,32957.0,...,58299.0,30771.0,174757.0,185750.0,623657.0,107849.0,42966.0,325196.0,326620.0,NaN
500530,500530,Chris Murphy,Deutscher Pomona,England,Man Utd,346623.0,80829.0,500119.0,528484.0,874874.0,...,900095.0,1578070.0,773255.0,1070078.0,251515.0,81188.0,19406.0,70036.0,1084997.0,NaN
500575,500575,Ross Stainsby,Harry's Boys,England,Leeds,2674477.0,3016352.0,4053942.0,2386281.0,3299849.0,...,1261573.0,844707.0,1012930.0,1924524.0,1253470.0,731606.0,458860.0,76066.0,179743.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499701,499701,Welshn White,All hail the Chief,Wales,Leeds,373835.0,3340461.0,860897.0,1356781.0,1017420.0,...,658955.0,327443.0,1138110.0,1216687.0,1062497.0,40409.0,223806.0,236631.0,11211.0,NaN
499768,499768,Ranieri Kulopp,Libang Libupul FC,Malaysia,Liverpool,2815175.0,2217134.0,113914.0,725700.0,1541747.0,...,517127.0,177977.0,1206042.0,1943327.0,875976.0,1330121.0,594271.0,756643.0,647133.0,NaN
499800,499800,huw montgomery,AC 8008135,England,Spurs,169751.0,1891325.0,3309396.0,1249696.0,97069.0,...,59802.0,1192737.0,32779.0,571821.0,211247.0,744215.0,274222.0,526591.0,229600.0,NaN
499916,499916,Daniel Shallicker,Shalski Devils,England,Man Utd,435655.0,140748.0,1590826.0,641296.0,1926625.0,...,325014.0,62940.0,70570.0,50910.0,74801.0,123595.0,56043.0,219865.0,43802.0,NaN
